# Artificial Neural Network Model

### IF3170 - Machine Learning

Developed by: 
1. Juan Christopher Santoso (13521116)
2. Nicholas Liem (13521135)
3. Nathania Calista Djunaedi (13521139)
4. Antonio Natthan Krishna (13521162)

#### **Import Library**

In [4]:
import numpy as np
import pandas as pd
import json
import copy

#### **Neural Network Properties**

#### 1. Activation Function

In [5]:
class ActivationFunction:
    def __init__(self, types='Sigmoid'):
        self.func = self.sigmoid
        self.dfunc = self.dsigmoid
        self.dfuncerr = self.dsum_square

        match types:
            case 'sigmoid':
                self.func = self.sigmoid
                self.dfunc = self.dsigmoid
                self.dfuncerr = self.dsum_square
            case 'linear':
                self.func = self.linear
                self.dfunc = self.dlinear
                self.dfuncerr = self.dsum_square
            case 'softmax':
                self.func = self.softmax
                self.dfunc = self.dsoftmax
                self.dfuncerr = self.derr_softmax
            case 'relu':
                self.func = self.relu
                self.dfunc = self.drelu
                self.dfuncerr = self.dsum_square

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def dsigmoid(self, x):
        sig = self.sigmoid(x)
        return sig * (1-sig)
    
    def linear(self, x):
        return x
    
    def dlinear(self, x):
        return 1
    
    def softmax(self, x):
        expX = np.exp(x - np.max(x, axis=1, keepdims=True))
        return expX / np.sum(expX, axis=1, keepdims=True)
    
    def relu(self, x):
        return np.maximum(0, x)
    
    def drelu(self, x):
        return np.where(x > 0, 1, 0)
    
    def dsum_square(self, output, target):
        return target - output
    
    def derr_softmax(self, output, target):
        if (target != 1):
            return output
        else:
            return output - 1

#### 2. Loss Function

In [6]:
class LossFunction:
    @staticmethod
    def calculate(output, target, layers):
        activation_mode = layers[-1]['activation_function']
        if (activation_mode == "softmax"):
            return LossFunction.loss_softmax(output, target)
        else:
            return LossFunction.loss_rsl(output, target)

    @staticmethod
    def loss_rsl(output, target):
        err = 0
        for i in range (len(output)):
            err += (target[i] - output[i])**2
        return err
    
    @staticmethod
    def loss_softmax(output, target):
        idx = np.where(target == 1)[0]
        return -1 * np.log10(output[idx])

#### 3. Forward Propagation (Using Fast Forward Neural Network)

In [7]:
class ForwardPropagation:
    @staticmethod
    def process(input_data, layers, weights):
        activations = input_data
        neuron_net = []
        neuron_out = []

        for i in range(len(layers)):
            activations_with_bias = np.insert(activations, 0, 1, axis=1)
            net_input = np.dot(activations_with_bias, weights[i])
            activation_mode = layers[i]['activation_function']
            activationFunc = ActivationFunction(activation_mode)
            activations = activationFunc.func(net_input)
            neuron_net.append(net_input)
            neuron_out.append(activations)

        return activations, neuron_net, neuron_out

#### 4. Backward Propagation

In [8]:
class BackwardPropagation:
    @staticmethod
    def process(weights, output, target, neuron_net, neuron_out, layers, learning_rate, input_data):
        delta = []

        # Delta Output Layer
        activation_mode = layers[-1]['activation_function']
        delta_layer = []
        activationFunc = ActivationFunction(activation_mode)
        if (activation_mode == 'softmax'):
            for i in range (len(output)): 
                delta_layer.append(activationFunc.dfuncerr(output[i], target[i]))
        else:
            for i in range (len(output)): 
                delta_layer.append(activationFunc.dfuncerr(output[i], target[i]) * activationFunc.dfunc(output[i]))
        delta.append(delta_layer)

        # Delta Hidden Layer
        for i in range(len(layers) - 2, -1, -1):
            activation_mode = layers[i]['activation_function']
            activationFunc = ActivationFunction(activation_mode)
            prev_delta = delta_layer if i == len(layers) - 2 else delta_layer[1:]
            delta_layer = []
            layer_weight = weights[i+1]
            layer_output = neuron_out[i][0]

            for j in range (len(layer_weight)): 
                neuron_weight = layer_weight[j]
                sigma = np.dot(neuron_weight, prev_delta)
                delta_layer.append(sigma * activationFunc.dfunc(1 if j == 0 else layer_output[j-1]))
            delta = [delta_layer[1:]] + delta

        # print(delta)

        # Update Weight
        for i in range (len(layers)):
            if (i == 0):
                layer_input = input_data
            else :
                layer_input = neuron_out[i-1][0]

            layer_input = np.insert(layer_input, 0, 1)
            for j in range (len(weights[i])):
                for k in range (len(weights[i][j])):
                    weights[i][j][k] += learning_rate * delta[i][k] * layer_input[j]
        
        return weights

#### **Artificial Neural Network**

In [9]:
class ArtificialNeuralNetwork:
    def __init__(self, architecture, input_data, output):
        self.layers = architecture.layers
        self.learning_rate = architecture.learning_rate
        self.error_threshold = architecture.error_threshold
        self.max_iter = architecture.max_iter
        self.batch_size = architecture.batch_size
        self.input_data = input_data.values
        self.output = output.values

        # Processing Target
        self.target = np.array([])
        for item in self.output:
            if item == "Iris-setosa":
                self.target = np.append(self.target, [1, 0, 0])
            elif item == "Iris-versicolor":
                self.target = np.append(self.target, [0, 1, 0])
            else:  # item == "Iris-virginica"
                self.target = np.append(self.target, [0, 0, 1])
        self.target = self.target.reshape(-1, 3)

        # Initializing Weight
        neurons_per_layer = [layer["number_of_neurons"] for layer in self.layers]
        self.weights = [[[1] * neurons_per_layer[0] for _ in range(len(self.input_data[0]) + 1)]]
        for i in range(1, len(neurons_per_layer)):
            prev_layer_neurons = neurons_per_layer[i - 1]
            current_layer_neurons = neurons_per_layer[i]
            self.weights.append([[1] * current_layer_neurons for _ in range(prev_layer_neurons + 1)])

    def train(self):
        temp_weight = copy.deepcopy(self.weights)

        for i in range (self.max_iter):
            error_total = 0
            minibatch = self.batch_size
            for i in range (len(self.input_data)):
                if (minibatch == 0):
                    self.weights = copy.deepcopy(temp_weight)
                    minibatch = self.batch_size
                output, neuron_net, neuron_out = ForwardPropagation.process([self.input_data[i]], self.layers, self.weights)
                error_total += LossFunction.calculate(output[0], self.target[i], self.layers)
                temp_weight = BackwardPropagation.process(temp_weight, output[0], self.target[i], neuron_net, neuron_out, self.layers, self.learning_rate, self.input_data[i])
                minibatch = minibatch - 1

            if (error_total < self.error_threshold):
                break
        self.weights = copy.deepcopy(temp_weight)
        
        print(self.weights)
        print(error_total)

    def predict(self, test):
        res = []
        for i in range (len(self.input_data)):
            output, _, _ = ForwardPropagation.process([test.values[i]], self.layers, self.weights)
            res.append(output)

        self.test = []
        for item in res:
            max_index = np.argmax(item)
            if (max_index == 0):
                self.test.append("Iris-setosa")
            elif (max_index == 1):
                self.test.append("Iris-versicolor")
            else:
                self.test.append("Iris-virginica")
        print(self.test)

#### **Preprocessing Data**

In [10]:
class JsonModelParser:
    def __init__(self, filepath):
        self.filepath = filepath
        self.data = self.load_json_file()
        self.parse_model_data()

    def load_json_file(self):
        try:
            with open(self.filepath, 'r', encoding='utf-8') as file:
                return json.load(file)
        except FileNotFoundError:
            print(f"The file {self.filepath} was not found")
            return None
        except json.JSONDecodeError:
            print(f"Error decoding JSON from the file {self.filepath}")
            return None

    def parse_model_data(self):
        if self.data:
            self.model = self.data.get('model', {})
            raw_layers = self.model.get('layers', [])   
            self.layers = [{'number_of_neurons': layer.get('number_of_neurons'),
                        'activation_function': layer.get('activation_function')}
                       for layer in raw_layers]
            self.learning_rate = self.model.get('learning_rate')
            self.error_threshold = self.model.get('error_threshold')
            self.max_iter = self.model.get('max_iter')
            self.batch_size = self.model.get('batch_size')

    @staticmethod
    def save_json_file(data, filepath):
        try:
            with open(filepath, 'w', encoding='utf-8') as file:
                json.dump(data, file, ensure_ascii=False, indent=4)
        except IOError:
            print(f"Could not save data to {filepath}")

In [11]:
df = pd.read_csv('iris.csv')
input = df.drop(columns=['Id', 'Species'])
target = df['Species']
architecture = JsonModelParser("arch_example.json")

#### **Processing Data**

In [12]:
model = ArtificialNeuralNetwork(architecture, input, target)
model.train()

In [ ]:
print(model.weights)
model.predict(input)


[[[0.7253086569122815, 0.7253086569122815, 0.7253086569122815], [-0.4112827310985512, -0.4112827310985512, -0.4112827310985512], [0.11524989843090758, 0.11524989843090758, 0.11524989843090758], [0.4133830755099026, 0.4133830755099026, 0.4133830755099026], [0.8628838393854685, 0.8628838393854685, 0.8628838393854685]], [[0.7822149361452975, 0.7822149361452975, 0.7822149361452975, 0.7822149361452975], [0.37693611886528805, 0.37693611886528805, 0.37693611886528805, 0.37693611886528805], [0.37693611886528805, 0.37693611886528805, 0.37693611886528805, 0.37693611886528805], [0.37693611886528805, 0.37693611886528805, 0.37693611886528805, 0.37693611886528805]], [[1.2001321446067714, 0.35574644283098045, -0.21421956238996828], [-0.11781815874836277, 0.0005528317068711164, 0.06437544127429624], [-0.11781815874836277, 0.0005528317068711164, 0.06437544127429624], [-0.11781815874836277, 0.0005528317068711164, 0.06437544127429624], [-0.11781815874836277, 0.0005528317068711164, 0.06437544127429624]]]
